In [1]:
# libraries
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
# Gaat het regenen of niet?

# TG        = Etmaalgemiddelde temperatuur (in 0.1 graden Celsius)
# FG        = Etmaalgemiddelde windsnelheid (in 0.1 m/s)
# UG        = Etmaalgemiddelde relatieve vochtigheid (in procenten)
# PG        = Etmaalgemiddelde luchtdruk herleid tot zeeniveau (in 0.1 hPa) berekend uit 24 uurwaarden
# RH        = Etmaalsom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm)

In [6]:
# import data
tempdf = pd.read_csv('Data.txt', low_memory=False)
tempdf['TG'] = pd.to_numeric(tempdf['TG'], errors='coerce')
tempdf['FG'] = pd.to_numeric(tempdf['FG'], errors='coerce')
tempdf['UG'] = pd.to_numeric(tempdf['UG'], errors='coerce')
tempdf['PG'] = pd.to_numeric(tempdf['PG'], errors='coerce')
tempdf['RH'] = pd.to_numeric(tempdf['RH'], errors='coerce')

df = tempdf[['TG','FG','UG','PG','RH']].dropna(how='all')

# ik heb gekozen om RH om te zetten naar 0 of 1. bij 0 regent het niet, bij 1 regent het wel.
df['RH'] = np.where(df['RH']<= 0, 0 ,1)

# deze df sla ik op om later de orginele waardes nog te kunnen gebruiken
originaldf = df

df.describe()

,TG,FG,UG,PG,RH
count,23825.000000,23824.000000,16559.000000,23829.000000,23829.000000
mean,99.013305,50.969862,82.332448,10150.417433,0.673885
std,59.214849,25.524074,8.557631,99.687823,0.468800
min,-122.000000,0.000000,35.000000,9614.000000,0.000000
25%,58.000000,31.000000,77.000000,10091.000000,0.000000
50%,100.000000,46.000000,83.000000,10157.000000,1.000000
75%,147.000000,67.000000,89.000000,10217.000000,1.000000
max,265.000000,195.000000,100.000000,10459.000000,1.000000


In [3]:
# Data cleanup
df['UG'] = df ['UG'].replace('     ',df['UG'].mean())
df['RH'] = df ['RH'].replace('     ',df['RH'].mean())

# ik heb gekozen om de null waardes te vervangen voor een gemiddelde omdat ik niet de gehele row wil laten vallen. 
# Aangezien daar veel bruikbare data in zit.
df['TG'] = df ['TG'].fillna(df['TG'].mean())
df['FG'] = df ['FG'].fillna(df['FG'].mean())
df['UG'] = df ['UG'].fillna(df['UG'].mean())
df['RH'] = df ['RH'].fillna(df['UG'].mean())

# normaliseren van de data
df_s = preprocessing.scale(df)
df_s = pd.DataFrame(df_s, columns = df.columns)
df = df_s

df.describe()

,TG,FG,UG,PG,RH
count,2.382900e+04,2.382900e+04,2.382900e+04,2.382900e+04,2.382900e+04
mean,2.385472e-17,-1.526702e-16,-1.561292e-15,2.972298e-15,-1.908378e-17
std,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00
min,-3.732788e+00,-1.997184e+00,-6.635205e+00,-5.381085e+00,-1.437498e+00
25%,-6.926913e-01,-7.824917e-01,-3.269695e-01,-5.960475e-01,-1.437498e+00
50%,1.666471e-02,-1.947373e-01,-1.992120e-15,6.603320e-02,6.956531e-01
75%,8.104678e-01,6.281190e-01,5.141286e-01,6.679248e-01,6.956531e-01
max,2.803420e+00,5.643624e+00,2.476691e+00,3.095554e+00,6.956531e-01


In [4]:
# split test and train data
X = df.drop(columns=['RH'])

# de y geef ik de 'orginele' waarde mee. deze bestaat uit 0 of 1.
y = originaldf['RH']

print(y)
# Ik heb gekozen voor een kleine test size omdat er veel data is.
# En omdat ik het trainen belangrijker vind dan het vergelijken van de test en train data.
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1)

0        1
1        1
2        1
3        1
4        1
        ..
23859    1
23860    1
23861    0
23862    0
23863    1
Name: RH, Length: 23829, dtype: int32


In [5]:
# setup en train model
model = Sequential()

# Ik heb gezien dat er van 50 naar 100 itteraties er WEINIG verbetering is in accuracy, bij het gebruik van dense 16, 8.
# epoch 1 - accuracy: 0.7121
# epoch 50 - accuracy: 0.8289
# epoch 100 - accuracy: 0.8316

# Ik heb gezien dat er vanaf 50 itteraties GEEN verbetering is in accuracy, bij het gebruik van dense 32, 16.
# Wat opvalt is dat er wel vanaf epoch 1 een hogere accuracy is in vergelijking tot een dense van 16, 8. 
# epoch 1 - accuracy: 0.7304
# epoch 50 - accuracy: 0.8328
# epoch 100 - accuracy: 0.8291


model.add(Dense(32, activation='relu', input_dim=4))
model.add(Dense(16, activation='relu', input_dim=4))

# Ik heb helaas geen wiskunde gestudeerd en zou niet kunnen uitleggen waarom sigmoid een goede keuzes is voor dit project.
model.add(Dense(1, activation='sigmoid'))

# Voor zo ver ik het gebrijp gebruik je Cross entropy om de berekenen hoeveel twee waarden van elkaar af liggen.
# In dit geval redeneer ik dat het gaat om de Binary afstand.
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy']  )

# train het model
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
671/671 [==============================] - 23s 1ms/step - loss: 0.5443 - accuracy: 0.7316
Epoch 2/100
671/671 [==============================] - 1s 1ms/step - loss: 0.4601 - accuracy: 0.7912
Epoch 3/100
671/671 [==============================] - 1s 2ms/step - loss: 0.4337 - accuracy: 0.7996
Epoch 4/100
671/671 [==============================] - 1s 1ms/step - loss: 0.4246 - accuracy: 0.8067
Epoch 5/100
671/671 [==============================] - 1s 1ms/step - loss: 0.4093 - accuracy: 0.8093
Epoch 6/100
671/671 [==============================] - 1s 2ms/step - loss: 0.4046 - accuracy: 0.8081
Epoch 7/100
671/671 [==============================] - 2s 3ms/step - loss: 0.3882 - accuracy: 0.8201
Epoch 8/100
671/671 [==============================] - 1s 2ms/step - loss: 0.3919 - accuracy: 0.8153
Epoch 9/100
671/671 [==============================] - 1s 2ms/step - loss: 0.3887 - accuracy: 0.8156
Epoch 10/100
671/671 [==============================] - 2s 4ms/step - loss: 0.3805 - accur

In [7]:
# test het model
score_test = model.evaluate(X_test, y_test)

75/75 [==============================] - 1s 2ms/step - loss: 0.3281 - accuracy: 0.8372


In [12]:
# vraag het netwerk om een voorspelling te doen. Gaat het regenen of niet?
# normale waardes van:
# TG tussen de  -122 en de 265.    -Met normalisatie  -3.72 / 2.80
# FG tussen de  0 en de 195.       -Met normalisatie  -1.99 / 5.64
# UG tussen de  35 en de 100.      -Met normalisatie  -6.64 / 2.48
# PG tussen de  9614 en de 10459.  -Met normalisatie  -5.38 / 3.10

# RH tussen de  -1 en de 675.  <== deze hoef je dus niet op te geven.

# geef het in deze volgorde op in de array: TG FG UG PG
# [[67,118,91,10128]]  op deze dag regende het. YYYYMMDD : 19751231
# [[65,67,89, 10282]]  op deze dag regende het niet YYYYMMDD : 19751227

X = [[67,118,91,10128]]

#dit heb ik nog niet werkend gkregen.
Xn = preprocessing.scale(X)

print(model.predict_classes(Xn))
#print(np.argmax(model.predict(Xn), axis=-1))

C:\Users\mrjor\AppData\Roaming\Python\Python39\site-packages\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
[[1]]
[0]
